In [31]:
import pandas as pd
from sqlalchemy import create_engine

# Put postgreSQL credeitials into sql_login.py
from sql_login import USERNAME as USERNAME
from sql_login import PASSWORD as PASSWORD

## Read from CSV

In [32]:
avocado_df = pd.read_csv('Resources/Avocado.csv')

In [33]:
avocado_df

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27318,2015-11-22,1.97,6249.43,1733.40,2873.92,30.95,1611.16,1590.00,21.16,0.0,organic,2015,WestTexNewMexico
27319,2015-11-29,2.08,4638.10,1395.02,2238.04,61.71,943.33,943.33,0.00,0.0,organic,2015,WestTexNewMexico
27320,2015-12-13,1.80,7836.65,2194.49,2981.01,25.97,2635.18,2598.45,36.73,0.0,organic,2015,WestTexNewMexico
27321,2015-12-20,1.92,6255.19,1512.45,2407.32,11.78,2323.64,2213.72,109.92,0.0,organic,2015,WestTexNewMexico


## Data Munging

In [34]:
# Get sales for volume sold on date
avocado_df['Total Sales'] = avocado_df['AveragePrice']*avocado_df['Total Volume']

In [35]:
# Drop unnecessary columns
avocado_df = avocado_df[['Date','Total Volume','Total Sales']]

# Sum total sales and volume on dates for all locations, not differentiating conventional/organic
avocado_gb = avocado_df.groupby(['Date'])
avocado_date_df = pd.DataFrame(avocado_gb.sum().reset_index())

avocado_date_df['price'] = avocado_date_df['Total Sales']/avocado_date_df['Total Volume']

# Extract year and quarter from date
a_date_df_len = avocado_date_df.shape[0]
year_list = [int(avocado_date_df.iloc[j,0][0:4]) for j in range(0,a_date_df_len)]
quarter_list = [(int(avocado_date_df.iloc[j,0][5:7])-1)//3+1 for j in range(0,a_date_df_len)]
id_list = [j for j in range(0,a_date_df_len)]

avocado_date_df['id'] = id_list
avocado_date_df['year'] = year_list
avocado_date_df['quarter'] = quarter_list

#for j in range(1,13):
#    print(j,(j-1)//3+1)

# Rename columns
avocado_date_df.columns = map(str.lower, avocado_date_df.columns)



avocado_date_df

,date,total volume,total sales,price,id,year,quarter
0,2015-01-04,8.467434e+07,8.107588e+07,0.957502,0,2015,1
1,2015-01-11,7.855581e+07,8.012434e+07,1.019967,1,2015,1
2,2015-01-18,7.838878e+07,8.188651e+07,1.044620,2,2015,1
3,2015-01-25,7.646628e+07,8.048259e+07,1.052524,3,2015,1
4,2015-02-01,1.194532e+08,1.078265e+08,0.902667,4,2015,1
...,...,...,...,...,...,...,...
248,2019-11-03,1.033671e+08,1.097273e+08,1.061531,248,2019,4
249,2019-11-10,1.073769e+08,1.111357e+08,1.035006,249,2019,4
250,2019-11-17,1.042417e+08,1.086276e+08,1.042074,250,2019,4
251,2019-11-24,9.521441e+07,1.029595e+08,1.081343,251,2019,4


## Connect to postgreSQL and import the dataframe

In [36]:
rds_connection_string = USERNAME+':'+PASSWORD+"@localhost:5432/avocados"
engine = create_engine(f'postgresql://{rds_connection_string}')
conn = engine.engine.connect()

In [37]:
# Drop avocado price table if it exists
conn.execute('DROP TABLE IF EXISTS avo_price;')

In [38]:
# Create table to hold daily avocado price data
conn.execute(
"""
    CREATE TABLE avo_price (
    id INT PRIMARY KEY,
    year INT,
    quarter INT,
    date date,
    price double precision);
"""
)

In [39]:
# Confirm tables
engine.table_names()

['cmg', 'avo_price']

In [40]:
avocado_date_df[['id','date','price']].to_sql(name='avo_price', con=engine, if_exists='append', index=False)

## Aggregate quarterly data

In [43]:
avocado_qtr_gb = avocado_date_df[['total volume','total sales','year','quarter']].groupby(['year','quarter'])
avocado_qtr_df = pd.DataFrame(avocado_qtr_gb.sum().reset_index())

id_list = [j for j in range(0,avocado_qtr_df.shape[0])]
avocado_qtr_df['id'] = id_list

avocado_qtr_df['price'] = avocado_qtr_df['total sales']/avocado_qtr_df['total volume']
avocado_qtr_df.drop(columns=['total volume','total sales'])

,year,quarter,id,price
0,2015,1,0,1.007266
1,2015,2,1,1.036020
2,2015,3,2,1.063065
3,2015,4,3,0.981179
4,2016,1,4,0.915574
5,2016,2,5,0.940609
6,2016,3,6,1.139525
7,2016,4,7,1.210259
8,2017,1,8,1.008010
9,2017,2,9,1.215040


In [44]:
# Drop avocado price table if it exists
conn.execute('DROP TABLE IF EXISTS avo_qtr_price;')

In [45]:
# Create table to hold avocado price data
conn.execute(
"""
    CREATE TABLE avo_qtr_price (
    id INT PRIMARY KEY,
    year INT,
    quarter INT,
    price double precision);
"""
)

In [46]:
avocado_date_df[['id','year','quarter','price']].to_sql(name='avo_qtr_price', con=engine, if_exists='append', index=False)